In [1]:
#Import Dependencies
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#import datasets
gdp = pd.read_csv("./resources/gdp.csv")
protest = pd.read_csv("./resources/world_protest.csv")

### Clean GDP dataframe
* convert from wide format to long format using pandas
* Rename column headers and reset index

*Note: For country/years with NaN, we decided to leave in NaN rather than convert to 0. NaN represents a null value for which data was not collected that year by the world bank. A value of 0 would represent a 0% GDP and could complicate downstream analysis*

In [3]:
gdp["id"] = gdp.index

In [4]:
gdp = pd.wide_to_long(gdp, ["X"], i = "id", j="year")

In [5]:
#Explain why we are leaving in the NaN
gdp_df = gdp.rename(columns= {"X":"gdp"}).reset_index()

In [6]:
gdp_df

,id,year,Country Code,Country Name,gdp
0,0,1961,ABW,Aruba,NaN
1,1,1961,AFG,Afghanistan,NaN
2,2,1961,AGO,Angola,NaN
3,3,1961,ALB,Albania,NaN
4,4,1961,AND,Andorra,NaN
...,...,...,...,...,...
15571,259,2019,XKX,Kosovo,4.171660
15572,260,2019,YEM,"Yemen, Rep.",NaN
15573,261,2019,ZAF,South Africa,0.152583
15574,262,2019,ZMB,Zambia,1.705793


## Clean Mass Protest dataframe

In [7]:
protest = protest[["id", 'country', 'year', 'protest']]
df = protest.groupby(['country', 'year']).sum()['protest'].reset_index()

,country,year,protest
0,Afghanistan,1990,0
1,Afghanistan,1991,1
2,Afghanistan,1992,0
3,Afghanistan,1993,0
4,Afghanistan,1994,0
